Source_Report

In [ ]:
import pandas as pd
import networkx as nx
import json
from datetime import datetime

 
file_path = ''
df = pd.read_excel(file_path)
 
G = nx.Graph()

for source in df['source'].dropna().unique():
     
    sub_df = df[df['source'] == source]
    
     
    names = sub_df['name'].tolist()
    ids = sub_df['ID'].tolist()
    
     
    for _, row in sub_df.iterrows():
         
        if isinstance(row['published_at'], datetime):
            time_str = row['published_at'].strftime('%Y-%m-%d %H:%M:%S')
        else:
            time_str = row['published_at']
        
        G.add_node(row['ID'], 
                   name=row['name'],
                   report=row['source'],
                   Source=row['Source'],
                   status=row['status'], 
                   time=time_str)
    
     
    for i in range(len(ids)):
        for j in range(i + 1, len(ids)):
            G.add_edge(ids[i], ids[j], source=source)

 
output_path = './ruby_graph.json'   
data = nx.node_link_data(G)   

 
def custom_json_serializer(obj):
    if isinstance(obj, datetime):
        return obj.strftime('%Y-%m-%d %H:%M:%S')
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')

with open(output_path, 'w') as f:
    json.dump(data, f, indent=4, default=custom_json_serializer)


In [ ]:
import os

 
subgraphs = list(nx.connected_components(G))

 
subgraphs_dir = ''
os.makedirs(subgraphs_dir, exist_ok=True)

 
for idx, component in enumerate(subgraphs):
    subgraph = G.subgraph(component)
    subgraph_data = nx.node_link_data(subgraph)
    
    subgraph_path = os.path.join(subgraphs_dir, f'subgraph_{idx + 1}.json')
    print(f'subgraph_{idx + 1}.json'+': '+ str(len(component)))
    with open(subgraph_path, 'w') as f:
        json.dump(subgraph_data, f, indent=4, default=custom_json_serializer)


WebCrawler_Report

In [ ]:
import pandas as pd
import networkx as nx
import json
import os

 
df1 = pd.read_excel('')
df2 = pd.read_excel('')

 
output_dir = ''
os.makedirs(output_dir, exist_ok=True)

 
graph_counter = 0

 
for idx, row in df1.iterrows():
    found_names = eval(row['found_names'])   
    report_url = row['url']

     
    G = nx.Graph()

     
    node_ids = []
    for name in found_names:
         
        match_row = df2[df2['name'] == name]
        
        if not match_row.empty:
             
            for _, match in match_row.iterrows():
                node_id = match['id']   
                node_attrs = match.to_dict()
                node_attrs['url'] = report_url   
                G.add_node(node_id, **node_attrs)
                node_ids.append(node_id)

     
    for i in range(len(node_ids)):
        for j in range(i + 1, len(node_ids)):
            G.add_edge(node_ids[i], node_ids[j], report=report_url)

     
    print(f"Graph {graph_counter}:")
    print(f"URL: {report_url}")
    print("Nodes:")
    for node_id in G.nodes(data=True):
        print(node_id)
    print("\n")

     
    output_path = os.path.join(output_dir, f'graph_{graph_counter}.json')
    output_data = nx.node_link_data(G)
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=4)

     
    graph_counter += 1
